# Generating photometry from a ``Galaxy``

If you are working with a ``Galaxy`` (or many ``Galaxy`` objects) containing multiple spectra, you can use the galaxy level ``get_photo_luminosities`` and ``get_photo_fluxes`` methods to generate photometry for all spectra in the galaxy.

We first load some CAMELS galaxies, and generate their rest--frame and observed spectra:

In [ ]:
from astropy.cosmology import Planck18 as cosmo
from synthesizer.emission_models import PacmanEmission
from synthesizer.emission_models.attenuation import PowerLaw
from synthesizer.emission_models.dust.emission import Greybody
from synthesizer.filters import FilterCollection
from synthesizer.grid import Grid
from synthesizer.load_data.load_camels import load_CAMELS_IllustrisTNG
from unyt import K

tau_v = 0.5
# dust curve slope
alpha = -1.0
dust_curve = PowerLaw(slope=alpha)
dust_emission_model = Greybody(30 * K, 1.2)

grid_dir = "../../../tests/test_grid"
grid_name = "test_grid"
grid = Grid(grid_name, grid_dir=grid_dir)

# Create galaxy object
gal = load_CAMELS_IllustrisTNG(
    "../../../tests/data/",
    snap_name="camels_snap.hdf5",
    fof_name="camels_subhalo.hdf5",
    physical=True,
)[0]

# Get the model
pacman = PacmanEmission(
    grid=grid,
    tau_v=tau_v,
    dust_curve=dust_curve,
    dust_emission=dust_emission_model,
)

# Get the spectra
sed = gal.stars.get_spectra(pacman)

# Get fluxes
gal.get_observed_spectra(cosmo)

We can then combine with a ``FilterCollection`` to generate the galaxy-level photometry.

In [ ]:

# Get the filter collection
fs = [
    f"JWST/NIRCam.{f}"
    for f in ["F090W", "F150W", "F200W", "F277W", "F356W", "F444W"]
]

filters = FilterCollection(
    filter_codes=fs,
    new_lam=grid.lam,
)

# Get the photometry
gal.get_photo_luminosities(filters)
gal.get_photo_fluxes(filters)

The photometry produced by these methods are stored in the ``photo_luminosities`` and ``photo_fluxes`` attributes, either at the base galaxy level or the individual components.
These attributes are dictionaries containing the photometry for each spectra key.

For example, on the ``Galaxy.Stars`` object:

In [ ]:
print(gal.stars.photo_luminosities)
print(gal.stars.photo_fluxes)

Or on the galaxy level

In [ ]:
print(gal.photo_luminosities)

As before we can print the photometry.

In [ ]:
print(gal.stars.photo_fluxes["nebular"])

Or plot them.

In [ ]:
gal.stars.photo_fluxes["nebular"].plot_photometry(show=True)

In [ ]:
gal.stars.photo_fluxes["intrinsic"].plot_photometry(show=True)

In [ ]:
gal.stars.photo_fluxes["total"].plot_photometry(show=True)